In [1]:
import scanpy as sc
import omicverse as ov
import scanpy.external as sce
ov.plot_set()


   ____            _     _    __                  
  / __ \____ ___  (_)___| |  / /__  _____________ 
 / / / / __ `__ \/ / ___/ | / / _ \/ ___/ ___/ _ \ 
/ /_/ / / / / / / / /__ | |/ /  __/ /  (__  )  __/ 
\____/_/ /_/ /_/_/\___/ |___/\___/_/  /____/\___/                                              

Version: 1.6.8, Tutorials: https://omicverse.readthedocs.io/
Dependency error: (pydeseq2 0.4.12 (/opt/anaconda3/envs/Omicverse/lib/python3.9/site-packages), Requirement.parse('pydeseq2<=0.4.0,>=0.3'))


In [2]:
adata = sc.read_h5ad("RAW.h5ad")
adata.var_names_make_unique()
adata.obs_names_make_unique()
adata

AnnData object with n_obs × n_vars = 310864 × 30048
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'group', 'dataset'

In [ ]:
adata=ov.pp.qc(adata,
              tresh={'mito_perc': 0.1, 'nUMIs': 1000, 'detected_genes': 500},
               doublets_method='sccomposite',
              batch_key="orig.ident")
adata

CPU mode activated
Calculate QC metrics
End calculation of QC metrics.
Original cell number: 310864
!!!It should be noted that the `sccomposite` will remove more cells than             `scrublet`!!!
Begin of post doublets removal and QC plot using `sccomposite`
Cuda is not available; Fitting the COMPOSITE model on RNA modality
The RNA modality goodness-of-fit score is: 5.013529788546683 
<3: poor fit 
3~5: moderate fit 
>5: good fit
Cuda is not available; Fitting the COMPOSITE model on RNA modality
The RNA modality goodness-of-fit score is: 4.751660972754328 
<3: poor fit 
3~5: moderate fit 
>5: good fit
Cuda is not available; Fitting the COMPOSITE model on RNA modality
The RNA modality goodness-of-fit score is: 4.28810410784142 
<3: poor fit 
3~5: moderate fit 
>5: good fit
Cuda is not available; Fitting the COMPOSITE model on RNA modality
The RNA modality goodness-of-fit score is: 4.383033259781744 
<3: poor fit 
3~5: moderate fit 
>5: good fit
Cuda is not available; Fitting the COMP

In [ ]:
ov.utils.store_layers(adata,layers="counts")
adata.layers["counts"]=adata.X.copy()

In [ ]:
adata.write_h5ad('/Volumes/WD/QCnod.h5ad',compression='gzip')

In [ ]:
adata=ov.pp.preprocess(adata,mode='shiftlog|seurat',
                       n_HVGs=2000,batch_key=None)
adata

In [ ]:
adata.raw = adata
adata = adata[:, adata.var.highly_variable_features]
adata

In [ ]:
ov.pp.scale(adata)
ov.pp.pca(adata,layer='scaled',n_pcs=50,mask_var='highly_variable_features')
adata

In [ ]:
adata

In [ ]:
adata_harmony=ov.single.batch_correction(adata,batch_key='orig.ident',
                                        methods='harmony',n_pcs=30)
adata

In [ ]:
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=30,
               use_rep='X_harmony')
sc.tl.umap(adata)
sc.tl.leiden(adata,resolution=0.3)

In [ ]:
ov.utils.embedding(adata,
                basis='X_umap',
                color=['leiden'],
                frameon='small',legend_loc="on data")

In [ ]:
marker_genes_dict = {
    "B_cells": ["CD79A","CD79B","MS4A1"],
    "Myeloid": ["LYZ","CD14","CSF1R","CD68"],
    'Mast_cells': ["TPSB2","TPSAB1","CPA3","MS4A2"],
    "Plasma_cells": ["MZB1","IGHG1","IGHG2"],
    "Epithelial_cells": ["KRT18","KRT19","KRT8"],
    "Fibroblast":["DCN","COL3A1","COL1A1","COL1A2","RGS5","ACTA2","MYH11"],
    "Endothelial": ["VWF","PECAM1","CLDN5","CDH5","FLT1"],
    "T-cell": ["CD3D","CD8A","CD3D","CD3E","CD2"],
    "Proliferation":["TOP2A","UBE2C","BIRC5"]}
sc.pl.dotplot(adata, marker_genes_dict, groupby='leiden', dendrogram=False, standard_scale='var',
              cmap='Spectral_r',save="markergene.png")

In [ ]:
cell_type_sub = {"0":"T_cell",
"1":"T_cell",
"2":"Myeloid",
"3":"Endothelial",
"4":"Epithelial",
"5":"Myeloid",
"6":"Fibroblast",
"7":"Epithelial",
"8":"Epithelial",
"9":"Myeloid",
"10":"Epithelial",
"11":"B_cell",
"12":"T_cell",
"13":"Mast_cell",
"14":"T_cell",
"15":"Endothelial",
"16":"Plasma",
"17":"Fibroblast",
"18":"Fibroblast",
"19":"Unclassified",
"20":"Unclassified"}
adata.obs['celltype'] = adata.obs.leiden.map(cell_type_sub)
ov.utils.embedding(adata,
                basis='X_umap',
                color=['celltype'],
                frameon='small',legend_loc="on data")

In [ ]:
adata_counts=adata.raw.to_adata().copy()
ov.utils.retrieve_layers(adata_counts,layers='counts')
print('normalize adata:',adata.X.max())
print('raw count adata:',adata_counts.X.max())
adata_counts

In [ ]:
adata_counts.write_h5ad('/Volumes/WD/CelltypeRaw.h5ad',compression='gzip')